# Initializing the SysML v2 API

In [138]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json
from datetime import datetime

#host = "<specify protocol://host:port of the server that is a provider of the SysML v2 REST/HTTP API"
host = "Specify the URL of the SysML v2 API and Service Repository"

# Get projects

In [139]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    projects_data = list(map(lambda b: {'Project Name':b['name'], 'Project ID':b['@id']}, projects))
    df = pd.DataFrame.from_records(projects_data).style.hide(axis='index')
    # if len(projects_data) > 0:
    #     df = df.style.sort_values(by='Project Name')
    display(df)
else:
    pprint("Problem in fetching projects")

Project Name,Project ID
12b-Allocation Fri Nov 04 19:01:21 EDT 2022,001c5431-b990-4e30-8c54-92f9a864ec97
13a-Model Containment Fri Nov 04 19:01:50 EDT 2022,02c58637-1259-4f3c-8239-93ceadc093a8
15_05-Unification of Expression and Constraint Definition Fri Nov 04 19:05:41 EDT 2022,0464d784-a8a9-4b83-b791-ea987f1d314f
15_10-Primitive Data Types Fri Nov 04 16:26:37 EDT 2022,0846c1fc-908a-4f31-b0a0-92b697ef9cbc
3a-Function-based Behavior-2 Fri Nov 04 16:10:36 EDT 2022,0ed6d350-8b64-430d-b51d-485f8593fc53
17b-Sequence-Modeling Fri Nov 04 19:07:56 EDT 2022,122c3e3d-f91b-47a6-80ad-88d4b9e80716
12b-Allocation Fri Nov 04 16:21:29 EDT 2022,13fb75cb-89f8-42b6-9f4e-b6d5bc28edf6
7b-Variant Configurations Fri Nov 04 16:15:18 EDT 2022,165b93d8-ba8f-4c1e-b2f1-440613ae81fe
2a-Parts Interconnection Fri Nov 04 16:09:04 EDT 2022,1b07b57e-7632-4da3-8d59-dc32442496f3
Spacecraft project with branches and tags - 2022-11-05 21:02:55.729180,1cb23e7a-4d70-487e-89ad-1cd8a22efb9e


# Create a new project

In [121]:
timestamp = datetime.now()
project_name = f"Spacecraft project with branches and tags - {timestamp}"
project_data = {
  "@type":"Project",
  "name": project_name,
  "description": "Spacecraft project with multiple commits, branches, and tags"
}

project_post_url = f"{host}/projects" 

project_post_response = requests.post(project_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(project_data))

project_id = ""

if project_post_response.status_code == 200:
    project_response_json = project_post_response.json()
    pprint(project_response_json)
    project_id = project_response_json['@id']
    project_name = project_response_json['name']
else:
    pprint(f"Problem in creating a new Spacecraft project at {timestamp}")
    pprint(project_post_response)

{'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e',
 '@type': 'Project',
 'created': '2022-11-05T21:02:55.930924-04:00',
 'defaultBranch': {'@id': 'b20df3d4-bb46-489f-b312-858952c21d89'},
 'description': 'Spacecraft project with multiple commits, branches, and tags',
 'name': 'Spacecraft project with branches and tags - 2022-11-05 '
         '21:02:55.729180'}


# Get branches

In [122]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches of Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
main,b20df3d4-bb46-489f-b312-858952c21d89,None


# Get tags

In [123]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    pprint(tags)
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags of Spacecraft project {project_id}")
    pprint(tags_response)

[]


# Create 1st Commit

In [124]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Spacecraft System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Payload System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Propulsion System"
      }
    }
  ]
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit1_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit1_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '68654706-7146-48f0-8ed9-ec0ffa94feea',
 '@type': 'Commit',
 'created': '2022-11-05T21:02:56.305306-04:00',
 'description': None,
 'owningProject': {'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e'},
 'previousCommit': None}


In [125]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit1_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit1_id}.")
    pprint(element_get_response)

Element Name,Element ID
Payload System,aa45735a-6001-4f00-a7f2-ecad5d442d27
Propulsion System,27e4f855-aa8a-4343-b25c-0c452cfb22ab
Spacecraft System,00734e2e-565f-4411-8b06-b5c8eab5e178


# Create 2nd Commit

In [126]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Avionics System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Power System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit1_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit2_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit2_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0',
 '@type': 'Commit',
 'created': '2022-11-05T21:02:56.7703-04:00',
 'description': None,
 'owningProject': {'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e'},
 'previousCommit': {'@id': '68654706-7146-48f0-8ed9-ec0ffa94feea'}}


# Get all elements after the 2nd commit

In [127]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}.")
    pprint(element_get_response)

Element Name,Element ID
Avionics System,d4a362f9-7a85-4887-9936-4df002c5fcae
Payload System,aa45735a-6001-4f00-a7f2-ecad5d442d27
Power System,8ac6138b-e00c-4908-8383-b25ef5dce6b0
Propulsion System,27e4f855-aa8a-4343-b25c-0c452cfb22ab
Spacecraft System,00734e2e-565f-4411-8b06-b5c8eab5e178


# Create a tag

In [128]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.1",
    "taggedCommit": {
      "@id": commit2_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '65b1977d-bc03-4a94-be41-2ed85eda65ed',
 '@type': 'Tag',
 'created': '2022-11-05T21:02:57.096895-04:00',
 'name': 'Spacecraft Internal Release 0.1',
 'owningProject': {'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e'},
 'referencedCommit': {'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'},
 'taggedCommit': {'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}}


# Get Tags

In [129]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.1,65b1977d-bc03-4a94-be41-2ed85eda65ed,{'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}


# Create a new branch at the second commit (also tagged as 0.1)

In [130]:
branch_body = {
  "@type": "Branch",
  "name": "develop",
  "head": {
      "@id": commit2_id
    }
}

branch_post_url = f"{host}/projects/{project_id}/branches" 

branch_post_response = requests.post(branch_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(branch_body))

branch_develop_id = ""

if branch_post_response.status_code == 200:
    branch_response_json = branch_post_response.json()
    pprint(branch_response_json)
    branch_develop_id = branch_response_json['@id']
else:
    pprint(f"Problem in creating a new branch in Spacecraft project {project_id}")
    pprint(branch_post_response)

{'@id': 'adb35786-0381-4f86-814e-deff94481ca5',
 '@type': 'Branch',
 'created': '2022-11-05T21:02:57.268637-04:00',
 'head': {'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'},
 'name': 'develop',
 'owningProject': {'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e'},
 'referencedCommit': {'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}}


# Get branches

In [131]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
develop,adb35786-0381-4f86-814e-deff94481ca5,{'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}
main,b20df3d4-bb46-489f-b312-858952c21d89,{'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}


# Create a new commit in the develop branch

In [132]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"GN & C System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit2_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body),
                                      params={"branchId":branch_develop_id})

commit3_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit3_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '5665c584-4474-4599-8b74-46c3c647a2cf',
 '@type': 'Commit',
 'created': '2022-11-05T21:02:57.444978-04:00',
 'description': None,
 'owningProject': {'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e'},
 'previousCommit': {'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}}


# Get Branches

In [133]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(commit_post_response)

Branch Name,Branch ID,Ref Commit (head)
develop,adb35786-0381-4f86-814e-deff94481ca5,{'@id': '5665c584-4474-4599-8b74-46c3c647a2cf'}
main,b20df3d4-bb46-489f-b312-858952c21d89,{'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}


# Get all elements as of the latest commit on main branch

In [134]:
pprint(f"Fetching elements at commit {commit2_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}")
    pprint(element_get_response)

'Fetching elements at commit 118e6f99-cde1-41fb-8f93-928fe6fa4be0'


Element Name,Element ID
Avionics System,d4a362f9-7a85-4887-9936-4df002c5fcae
Payload System,aa45735a-6001-4f00-a7f2-ecad5d442d27
Power System,8ac6138b-e00c-4908-8383-b25ef5dce6b0
Propulsion System,27e4f855-aa8a-4343-b25c-0c452cfb22ab
Spacecraft System,00734e2e-565f-4411-8b06-b5c8eab5e178


# Get all elements as of the latest commit on develop branch

In [135]:
pprint(f"Fetching elements at commit {commit3_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit3_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit3_id}")
    pprint(element_get_response)

'Fetching elements at commit 5665c584-4474-4599-8b74-46c3c647a2cf'


Element Name,Element ID
Avionics System,d4a362f9-7a85-4887-9936-4df002c5fcae
GN & C System,27d2c686-7c67-4cc5-99f1-cadc06cd4347
Payload System,aa45735a-6001-4f00-a7f2-ecad5d442d27
Power System,8ac6138b-e00c-4908-8383-b25ef5dce6b0
Propulsion System,27e4f855-aa8a-4343-b25c-0c452cfb22ab
Spacecraft System,00734e2e-565f-4411-8b06-b5c8eab5e178


# Create a tag at the last commit on develop branch

In [136]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.2 build 1",
    "taggedCommit": {
      "@id": commit3_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '040ddf64-13a9-4bf8-b533-ad831f7862db',
 '@type': 'Tag',
 'created': '2022-11-05T21:02:58.049038-04:00',
 'name': 'Spacecraft Internal Release 0.2 build 1',
 'owningProject': {'@id': '1cb23e7a-4d70-487e-89ad-1cd8a22efb9e'},
 'referencedCommit': {'@id': '5665c584-4474-4599-8b74-46c3c647a2cf'},
 'taggedCommit': {'@id': '5665c584-4474-4599-8b74-46c3c647a2cf'}}


# Get all tags

In [137]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.2 build 1,040ddf64-13a9-4bf8-b533-ad831f7862db,{'@id': '5665c584-4474-4599-8b74-46c3c647a2cf'}
Spacecraft Internal Release 0.1,65b1977d-bc03-4a94-be41-2ed85eda65ed,{'@id': '118e6f99-cde1-41fb-8f93-928fe6fa4be0'}
